In [45]:
import pandas as pd

In [46]:
log = pd.read_csv('log.csv', header = None)
log.columns = ['user_id', 'time', 'bet', 'win']

users = pd.read_csv('users.csv', sep='\t', encoding = 'koi8-r')
users.columns = ['user_id', 'email', 'geo']

def correct_user_id(incorrect_user_id):
    return incorrect_user_id.split('Запись пользователя № - ')[1] if not '#error' in incorrect_user_id else ''
log['user_id'] = log['user_id'].apply(correct_user_id)
log['time'] = log['time'].str.strip('[')
log['time'] = pd.to_datetime(log['time'])

log = log.dropna(axis=0, subset=['user_id', 'time'])
#log = log.drop_duplicates(subset=['user_id', 'time'])

log['bet'] = log["bet"].fillna(0)
log['win'] = log['win'].fillna(0)

def fillna_win(row):
    if row.win>0:
        return row.win
    elif row.bet==0:
        row.win=0
        return row.win
    elif row.bet>0 and row.win==0:
        return row.win-row.bet  

log['win'] = log.apply(lambda row: fillna_win(row), axis=1)

log['net'] = log['win'] - log['bet']

users['user_id'] = users['user_id'].apply(lambda x: x.lower())

In [47]:
log

,user_id,time,bet,win,net
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0
1,user_973,2019-01-01 14:51:16,0.0,0.0,0.0
2,user_903,2019-01-01 16:31:16,0.0,0.0,0.0
3,user_954,2019-01-01 17:17:51,0.0,0.0,0.0
4,user_954,2019-01-01 21:31:18,0.0,0.0,0.0
...,...,...,...,...,...
991,user_965,2019-04-20 12:55:41,800.0,6927.0,6127.0
992,user_967,2019-04-20 14:59:36,10154.0,-10154.0,-20308.0
993,user_973,2019-04-20 17:09:56,10254.0,-10254.0,-20508.0
994,user_977,2019-04-20 18:10:07,10354.0,-10354.0,-20708.0


In [48]:
users

,user_id,email,geo
0,user_943,Accumanst@gmail.com,Ижевск
1,user_908,Advismowr@mail.ru,Ижевск
2,user_962,Anachso@ukr.net,Краснодар
3,user_973,Antecia@inbox.ru,Пермь
4,user_902,Balliaryva@ukr.net,NaN
...,...,...,...
95,user_959,UpdatesCurious@yahoo.com,Тюмень
96,user_901,V2artierso@mail.ru,Арзангелтск
97,user_970,Vashoterlo@bk.ru,Воронеж
98,user_965,Visuareda@yahoo.com,Воронеж


In [49]:
merged_df = pd.merge(log, users, on='user_id')

In [50]:
merged_df.groupby('user_id')['net'].sum().median()

-1800.0